### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [1]:
import pandas as pd
import os
import tools as tools
from pyspark.sql import SparkSession, DataFrame

#### Immigration Dataset

In [2]:
df_immi_sample = pd.read_csv('immigration_data_sample.csv')

#### Countries Dataset

In [3]:
df_countryCodes = pd.read_csv('countries.csv')

#### Ports Dataset

In [4]:
portCodes = pd.read_csv('ports.csv')

#### Demographics Dataset

In [9]:
df_demographics = pd.read_csv('us-cities-demographics.csv', sep=';')

#### Airports Dataset

In [6]:
df_airport = pd.read_csv('airport-codes_csv.csv')

#### World Temperature Data

In [7]:
# Performing cleaning tasks here
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(fname)

In [8]:
df_temperature.shape

(8599212, 7)

In [10]:
len(df_temperature['Country'].unique())

159

In [11]:
# Just get temperature of USA
df_temperature = df_temperature[df_temperature['Country']=='United States'].copy()

In [12]:
df_temperature.shape

(687289, 7)

In [13]:
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [14]:
df_temperature.isnull().sum()

dt                                   0
AverageTemperature               25765
AverageTemperatureUncertainty    25765
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [15]:
# get the data is Null or NaN with AverageTemperature & AverageTemperatureUncertainty
df_temp_null = df_temperature[df_temperature.AverageTemperature.isnull()]
df_temp_null.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47723,1834-01-01,NaN,NaN,Abilene,United States,32.95N,100.53W
137067,1743-12-01,NaN,NaN,Akron,United States,40.99N,80.95W
137068,1744-01-01,NaN,NaN,Akron,United States,40.99N,80.95W
137069,1744-02-01,NaN,NaN,Akron,United States,40.99N,80.95W
137070,1744-03-01,NaN,NaN,Akron,United States,40.99N,80.95W


In [16]:
# drop all Null And NaN data.
df_drop = df_temperature[df_temperature.AverageTemperature.isnull() & df_temperature.AverageTemperatureUncertainty.isnull()]
df_drop.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47723,1834-01-01,NaN,NaN,Abilene,United States,32.95N,100.53W
137067,1743-12-01,NaN,NaN,Akron,United States,40.99N,80.95W
137068,1744-01-01,NaN,NaN,Akron,United States,40.99N,80.95W
137069,1744-02-01,NaN,NaN,Akron,United States,40.99N,80.95W
137070,1744-03-01,NaN,NaN,Akron,United States,40.99N,80.95W


In [19]:
df_temperature = df_temperature.drop(df_temperature[df_temperature.AverageTemperature.isnull() & df_temperature.AverageTemperatureUncertainty.isnull()].index, inplace = False)

In [20]:
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [21]:
# Drop duplicate data
duplicateRows = df_temperature[df_temperature.duplicated()]
duplicateRows.shape

(0, 7)

#### Cleaning the Airport Data

In [22]:
df_airport.shape

(55075, 12)

In [23]:
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [24]:
df_airport.isnull().sum()

ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

Check the countries where these airports are located

In [25]:
df_airport.groupby('iso_country')['iso_country'].count()

iso_country
AD        2
AE       57
AF       64
AG        3
AI        1
AL       13
AM       13
AO      104
AQ       27
AR      848
AS        4
AT      145
AU     1963
AW        1
AZ       35
BA       15
BB        6
BD       16
BE      146
BF       51
BG      134
BH        4
BI        7
BJ       10
BL        1
BM        3
BN        2
BO      197
BQ        3
BR     4334
      ...  
TM       21
TN       15
TO        6
TR      124
TT        3
TV        3
TW       65
TZ      207
UA      191
UG       38
UM        6
US    22757
UY       54
UZ      176
VA        1
VC        6
VE      592
VG        3
VI        9
VN       50
VU       32
WF        2
WS        4
XK        6
YE       25
YT        1
ZA      489
ZM      103
ZW      138
ZZ        7
Name: iso_country, Length: 243, dtype: int64

In [26]:
# Check type of airports
df_airport['type'].unique().tolist()

['heliport',
 'small_airport',
 'closed',
 'seaplane_base',
 'balloonport',
 'medium_airport',
 'large_airport']

In [27]:
# Verify the municipality field is available for all airports
df_airport[df_airport.municipality.isna()].head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
1544,15SD,heliport,Watertown / Brownlee Heliport,1720.0,NaN,US,US-SD,NaN,15SD,NaN,NaN,"-97.1080899239, 44.883264878199995"
2452,21ID,heliport,Nordman / Phillabaum Heliport,2440.0,NaN,US,US-ID,NaN,21ID,NaN,NaN,"-116.871174574, 48.631483378700004"
4401,3ME7,seaplane_base,Peru / Destiny Cove SPB,580.0,NaN,US,US-ME,NaN,3ME7,NaN,NaN,"-70.396957, 44.460597"
7653,6XA4,small_airport,Zadow Airstrip,NaN,NaN,US,US-TX,NaN,6XA4,NaN,NaN,"-95.954353809, 29.991738550900003"
7887,74xa,small_airport,Gun Barrel City Airpark,385.0,NaN,US,US-TX,NaN,74XA,NaN,NaN,"-96.1456650496, 32.3551499558"


In [30]:
# Clean all the municipality field that null
df_airport = df_airport[~df_airport['municipality'].isna()].copy()
df_airport.municipality = df_airport.municipality.str.upper().str.strip()
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,BENSALEM,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,LEOTI,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,ANCHOR POINT,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,HARVEST,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,NEWPORT,NaN,NaN,NaN,"-91.254898, 35.6087"


In [31]:
df_airport.groupby('iso_region')['iso_region'].count()

iso_region
AD-04      1
AD-07      1
AE-AZ     15
AE-DU     11
AE-RK      1
AE-SH      1
AF-BAM     1
AF-BDG     1
AF-BDS     5
AF-FRA     1
AF-FYB     1
AF-GHA     2
AF-GHO     1
AF-HEL     2
AF-HER     1
AF-KAB     1
AF-NIM     1
AF-ORU     3
AF-PAR     1
AF-PIA     2
AF-PKA     2
AF-TAK     2
AG-03      1
AG-10      2
AI-U-A     1
AL-01      1
AL-05      1
AL-06      1
AL-07      1
AL-08      1
          ..
ZA-FS     48
ZA-GT     36
ZA-MP     50
ZA-NC     76
ZA-NL     58
ZA-NP     59
ZA-NW     32
ZA-U-A     1
ZA-WC     44
ZM-01     10
ZM-02     10
ZM-03      9
ZM-04      7
ZM-05     14
ZM-06     12
ZM-07     15
ZM-08      8
ZM-09     11
ZM-U-A     1
ZW-BU      1
ZW-HA      2
ZW-MA     16
ZW-MC      7
ZW-ME      7
ZW-MI      8
ZW-MN     18
ZW-MS     10
ZW-MV     14
ZW-MW     21
ZW-U-A     1
Name: iso_region, Length: 2643, dtype: int64

#### Cleaning the US Cities Demographics

In [33]:
df_demographics.shape

(2891, 12)

In [32]:
df_demographics.isnull().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [35]:
df_demographics['City'] = df_demographics['City'].str.strip().str.upper()
df_demographics['State'] = df_demographics['State'].str.strip().str.upper()
df_demographics['Race'] = df_demographics['Race'].str.strip().str.upper()
df_demographics['State Code'] = df_demographics['State Code'].str.strip()

In [36]:
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,SILVER SPRING,MARYLAND,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,HISPANIC OR LATINO,25924
1,QUINCY,MASSACHUSETTS,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,WHITE,58723
2,HOOVER,ALABAMA,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,ASIAN,4759
3,RANCHO CUCAMONGA,CALIFORNIA,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,BLACK OR AFRICAN-AMERICAN,24437
4,NEWARK,NEW JERSEY,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,WHITE,76402


In [37]:
# found duplicate Row
duplicateRows = df_demographics[df_demographics.duplicated(['City', 'Race'])]
check_df = df_demographics[(df_demographics['City'] == 'WILMINGTON') & (df_demographics['Race'] == 'ASIAN')]
check_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
102,WILMINGTON,NORTH CAROLINA,35.5,52346.0,63601.0,115947,5908.0,7401.0,2.24,NC,ASIAN,3152
177,WILMINGTON,DELAWARE,36.4,32680.0,39277.0,71957,3063.0,3336.0,2.45,DE,ASIAN,1193


In [38]:
# Let's check with City, Race, State and no found duplicate
duplicateRows = df_demographics[df_demographics.duplicated(['City', 'Race', 'State'])]
duplicateRows.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


#### Cleaning the Immigration

In [53]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [54]:
df_spark.count()

3096313

In [55]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [42]:
df_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [56]:
df_spark.createOrReplaceTempView("immigration_table")

In [57]:
spark.sql("""
SELECT COUNT (DISTINCT cicid)
FROM immigration_table
""").show()

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|              3096313|
+---------------------+



In [58]:
# All dates in SAS correspond to the number of days since 1960-01-01. Therfore, we compute the arrival dates by adding arrdate to 1960-01-01
spark.sql("""
SELECT *, 
date_add(to_date('1960-01-01'), arrdate) AS arrival_date 
FROM immigration_table
""").createOrReplaceTempView("immigration_table")

spark.sql("""
SELECT *, 
CASE 
    WHEN depdate >= 1.0 THEN date_add(to_date('1960-01-01'), depdate)
    WHEN depdate IS NULL THEN NULL
    ELSE 'N/A' END AS departure_date 
FROM immigration_table""").createOrReplaceTempView("immigration_table")

In [59]:
# Show all departure_date < arrival_date
spark.sql("""
SELECT COUNT(*) 
FROM immigration_table 
WHERE departure_date < arrival_date
""").show()

+--------+
|count(1)|
+--------+
|     375|
+--------+



In [61]:
spark.sql("""
SELECT arrival_date, departure_date
FROM immigration_table
WHERE departure_date < arrival_date
""").show(10)

+------------+--------------+
|arrival_date|departure_date|
+------------+--------------+
|  2016-04-01|    2016-03-31|
|  2016-04-02|    2016-03-19|
|  2016-04-02|    2016-01-26|
|  2016-04-02|    2016-04-01|
|  2016-04-02|    2016-01-31|
|  2016-04-02|    2016-04-01|
|  2016-04-03|    2016-04-02|
|  2016-04-04|    2016-03-12|
|  2016-04-05|    2016-04-04|
|  2016-04-05|    2016-04-04|
+------------+--------------+
only showing top 10 rows



In [66]:
# dop them
spark.sql("""
SELECT *
FROM immigration_table
WHERE departure_date >= arrival_date
""").createOrReplaceTempView("immigration_table")

In [67]:
# Check the gender
spark.sql("""
SELECT gender, count(*) 
FROM immigration_table
GROUP BY gender
""").show()

+------+--------+
|gender|count(1)|
+------+--------+
|     F| 1228646|
|  null|  407456|
|     M| 1316305|
|     U|     238|
|     X|     836|
+------+--------+



In [68]:
# Check the visa type
spark.sql("""
SELECT i94visa, visatype, count(*)
FROM immigration_table
GROUP BY i94visa, visatype
ORDER BY i94visa, visatype
""").show()

+-------+--------+--------+
|i94visa|visatype|count(1)|
+-------+--------+--------+
|    1.0|      B1|  206339|
|    1.0|      E1|    3183|
|    1.0|      E2|   16232|
|    1.0|     GMB|     132|
|    1.0|       I|    2962|
|    1.0|      I1|     214|
|    1.0|      WB|  279702|
|    2.0|      B2| 1046528|
|    2.0|      CP|   12078|
|    2.0|     CPL|       8|
|    2.0|     GMT|   79883|
|    2.0|     SBP|       2|
|    2.0|      WT| 1275913|
|    3.0|      F1|   27792|
|    3.0|      F2|    1774|
|    3.0|      M1|     709|
|    3.0|      M2|      30|
+-------+--------+--------+

